# Parker Dunn
pgdunn@bu.edu | pdunn91@gmail.com

Created on July 1st, 2022
Finished on August 5th, 2022


__Assignment for COURSERA: Introduction to Deep Learning (via CU Boulder)__

__Assignment:__ Week 3 - CNN Cancer Detection Kaggle Mini-Project

# Step 1 - Description of the data and problem

## First, here are all imports and some important variables

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Working with files
import os
import shutil
import glob
import pathlib

# Data manipulation and linear algebra packages
import numpy as np
import pandas as pd

# Visualization and image packages
from skimage import io
import matplotlib.pyplot as plt

# TensorFlow imports
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

# Other untility packages
import json
from datetime import datetime

# PULL THE KAGGLE COMPEITION DATA IN
# For this notebook, made on Kaggle, I can import the data manually.
# All data ends up in "/kaggle/input/"

        
PATH="/kaggle/working"
INPUT="/kaggle/input/histopathologic-cancer-detection"
INPUT2="/kaggle/input/histopathologic-cancer-detection-supplement"

TRAIN=pathlib.Path("/kaggle/working/train/")
TEST=pathlib.Path("/kaggle/input/histopathologic-cancer-detection/test/")

TEST_LOCS=os.listdir(TEST)

batch_size=64
img_dim=96

%matplotlib inline

## Description of the data and problem

For this project, originally a Kaggle competition, the task is to identify images of cells that contain metastatic cancer. This is a binary classification task on images. If an image contains at least one pixel of tumor tissue within a specified region, it is given a positive label. The images are 96x96 pixels, but the specific region used to classify the images is the center 32x32 pixel region. The task is setup this way to enable fully-convolutional models that do not use padding.

The data for this task is provided as 96x96 images, which are small parts of larger images. Reading in the three color channels (RGB) for each pixel, this means there are 27,648 total features for each image: the three RGB channels for each pixel and 9,216 pixels. Each pixel is provided as three values from 0 to 255, representing the contribution of each RGB color channel.

There are 277,483 images provided for this task. The images are divided into 220,025 images for training, since the corresponding labels are provided, and 57,458 images without provided labels. For these test images, a model will try to correctly predict whether these images have (1) or do not have (0) metastatic cancer cells.

Summary:
* 96x96x3 dimensions for each image -> 27,648 features initially
* 220,025 images in training set
* TRAINING: 27648 features x 220,025 images

## At the end of step 1, I am defining some functions that I will use and loading information about the images/data.

In [12]:
def load_image_info(PATH):    
    train_locs = glob.glob(f"{PATH}/train/*.tif")
    test_locs = glob.glob(f"{PATH}/test/*.tif")
    
    num_train = len(train_locs)
    num_test = len(test_locs)
    
    y_train = pd.read_csv(f"{PATH}/train_labels.csv", header=0)
    
    return train_locs, test_locs, y_train


In [13]:
training_images, testing_images, y_train = load_image_info(INPUT)

y_train.head(8)

print("Double checking the type of the 'id' of the images...",
      "\n\n",
      y_train.loc[0,'id'],
      "\n\n",
      type(y_train.loc[0,'id']))

# Step 2 - Exploratory Data Analysis (EDA)

## Starting off by writing some functions that I'll use during EDA

In [14]:
def show_training_image(PATH, img_info):
    # displaying the image
    file = PATH+"/train/"+img_info.loc["id"]+".tif"
    image = io.imread(file)
    plt.imshow(image)
    plt.title("{}\n Class: {}".format(img_info.loc["id"], img_info.loc["label"]))
    
    # Drawing the center 32x32 region on the picture
    rectangle = plt.Rectangle((32,32), 32, 32, ec="red", linewidth=1.5, fill=False)
    plt.gca().add_patch(rectangle)
    plt.legend(["Classification Region"])
    
def show_training_images(PATH, img_info, dim):
    fig, axes = plt.subplots(nrows=dim[0], ncols=dim[1], figsize=(10,8))
    
    ax = axes.flatten()
    plt.subplots_adjust(hspace=0.4)
    
    for i in range(len(img_info.index)):
        #file = "train/" + img_info.loc[i,"id"] + ".tif"
        file = PATH+"/train/"+img_info.loc[i,"id"]+".tif"
        image = io.imread(file)
        ax[i].imshow(image)
        ax[i].set_title("{}\n Class: {}".format(img_info.loc[i,"id"], img_info.loc[i,"label"]))
        rectangle = plt.Rectangle((32,32), 32, 32, ec="red", linewidth=1.5, fill=False)
        ax[i].add_patch(rectangle)
        ax[i].legend(["Classification Region"])

def load_image_data(PATH, img):
    file = PATH+"/train/"+img+".tif"
    image = io.imread(file)
    return image

def calculate_img_avgs(images):
    avg = np.zeros((96,96,3))
    for img in images:
        image_np = load_image_data(img)
        avg = avg + image_np
    avg = avg/len(images)
    return avg

## Inspecting a single image


In [15]:
print(y_train.iloc[0,:])
print(type(y_train.iloc[0,:]),"\n\n")

show_training_image(INPUT, y_train.iloc[0,:])

## Inspecting multiple images

In [16]:
show_training_images(INPUT, y_train.iloc[0:4,:], (2,2))

## Examining information distribution of the images

Here, I looked at...
* the "average" picture
* comparison of purple primary color distributions

### Average of all training images

Originally, I used the code below to generate the average image data.

```python
# loop to collect averages of 100 images at a time
# the 100 image avgs are saved in a list
avg_img_100 = []
moving_avg = np.zeros((96,96,3))
counter = 0

for i in range(len(y_train.index)):
    if (i % 100 == 0) and (i != 0):
        moving_avg = moving_avg/100
        avg_img_100.append(moving_avg)
        moving_avg = np.zeros((96,96,3))
        counter = 0
    if (i % 10000 == 0):
        print("Progress...")
    
    image = load_image_data(y_train.loc[i,"id"])
    counter += 1
    moving_avg = moving_avg + image

if counter > 0:
    print("End value of counter: ",counter)
    avg_img_100.append(moving_avg/counter)

print("Number of NumPy arrays saved in avg_img_100: ", len(avg_img_100))
# print(avg_img_100[0].shape)

# getting the "average image" from the averages of 100 images
avg_img = np.zeros((96,96,3))
for i in range(len(avg_img_100)):
    avg_img = avg_img + avg_img_100[i]

avg_img = avg_img/len(avg_img_100)

savable_avg_img = avg_img.reshape(96*96,3)

np.savetxt("/kaggle/working/avg_training_image.txt", savable_avg_img, delimiter=",")

print("Shape of 'avg_img': ", avg_img.shape)
print("\nSample values...\n", avg_img[0:2,0:2,:])

```

The code above is very slow. It works to get the information desired, but running it multiple times makes no sense. Since I saved the resulting information in a text file, I will upload the data to this environment and load it from the saved file rather than putting the above code in a code cell and running it again.

In [17]:
# Loading the text file data
avg_img = np.genfromtxt(f"{INPUT2}/avg_training_image.txt", delimiter=',')

# Transforming the image shape
avg_img = avg_img.reshape(96,96,3)

# Displaying the avg image
avg_img_int = avg_img.astype('int')
plt.imshow(avg_img_int)
plt.title("Average training image - Both Classes")

I probably should have figured that the avg of all images would not be particularly helpful.

I am curious to see if there is a difference between the average positive vs. negative image. I will basically repeat the same process once more; hopefully, there is some more useful information there.

### Average image with positive and negative images separated (& additionally tracking the average R and B channel values across each image)

Once again, below is the code that I originally used. However, I am going to load the image data that I saved rather than running this code again.

```python
# loop to collect averages of 100 images at a time
# the 100 image avgs are saved in a list
avg_img_pos = []
avg_img_neg = []
moving_avg_pos = np.zeros((96,96,3))
moving_avg_neg = np.zeros((96,96,3))
counter_pos = 0
counter_neg = 0

# Simultaneous task -> track and save values for R & B channels of each image
pos_R = []
pos_B = []
neg_R = []
neg_B = []
# End of setup for simultaneous task

for i in range(len(y_train.index)):
    if (counter_pos == 1000):
        moving_avg_pos = moving_avg_pos/counter_pos
        avg_img_pos.append(moving_avg_pos)
        moving_avg_pos = np.zeros((96,96,3))
        counter_pos = 0
    
    if (counter_neg == 1000):
        moving_avg_neg = moving_avg_neg/counter_neg
        avg_img_neg.append(moving_avg_neg)
        moving_avg_neg = np.zeros((96,96,3))
        counter_neg = 0
    
    image = load_image_data(y_train.loc[i,"id"])
    #print(image[:,:,0].shape)
    
    if (y_train.loc[i,"label"] == 1):
        counter_pos += 1
        moving_avg_pos = moving_avg_pos + image
        
        avg_R = image[:,:,0].reshape((1,-1)).mean()
        avg_B = image[:,:,2].reshape((1,-1)).mean()
        pos_R.append(avg_R)
        pos_B.append(avg_B)
    else:
        counter_neg += 1
        moving_avg_neg = moving_avg_neg + image
        
        avg_R = image[:,:,0].reshape((1,-1)).mean()
        avg_B = image[:,:,2].reshape((1,-1)).mean()
        neg_R.append(avg_R)
        neg_B.append(avg_B)

if (counter_pos > 0):
    avg_img_pos.append(moving_avg_pos/counter_pos)
if (counter_neg > 0):
    avg_img_neg.append(moving_avg_neg/counter_neg)

avg_pos = np.zeros((96,96,3))
avg_neg = np.zeros((96,96,3))

# POSITIVE IMAGES
for i in range(len(avg_img_pos)):
    avg_pos = avg_pos + avg_img_pos[i]
avg_pos = avg_pos/len(avg_img_pos)

# NEGATIVE IMAGES
for j in range(len(avg_img_neg)):
    avg_neg = avg_neg + avg_img_neg[i]
avg_neg = avg_neg/len(avg_img_neg)

for img, file in zip([avg_pos, avg_neg], ["average_positive_image", "average_negative_image"]):
    print(file, img.shape)
    np.save(f"/kaggle/working/{file}", img)
```

The images below are the average images by pixel across all of the negative and positive images.

In [18]:
# Loading the average pos and neg images
avg_pos_img = np.load(f"{INPUT2}/average_positive_image.npy")
avg_neg_img = np.load(f"{INPUT2}/average_negative_image.npy")

# Showing the images
fig, axes = plt.subplots(ncols=2, nrows=1, figsize=(12,10), sharey=True)

avg_pos_int = avg_pos_img.astype('int')
avg_neg_int = avg_neg_img.astype('int')

axes[0].imshow(avg_pos_int)
axes[0].set_title("Avg Positive Image")
axes[1].imshow(avg_neg_int)
axes[1].set_title("Avg Negative Image")

There isn't much to take away from these images. Since they are averages over many images, they look mostly uniform.

There is a slight difference in the shade of purple and presentation of cell shapes. The shade of purple of the average negative images appears to be slightly darker and more transparent. The average negative image also appears to have some darker spots that are faintly identfiable. Since unhealthy cancer cells lose their distinct oval/circular shape, this indicates the shape of the healthy cells is a fundamental feature of the images that can help me distiguish between the two classes of images.

__Clearly, purple is the dominant color of these images (due to the staining process used for visualization). Next, I will compare how the component colors of purple (red and blue) compare between the positive and negative images.__

### Looking at R and B channel distributions in positive and negative images

In [19]:
# LOADING THE IMAGE DATA
pos_images_channel_vals = pd.read_csv(f"{INPUT2}/positive_images_channel_vals.csv")
neg_images_channel_vals = pd.read_csv(f"{INPUT2}/negative_images_channel_vals.csv")

pos_R = pos_images_channel_vals.R
pos_B = pos_images_channel_vals.B
neg_R = neg_images_channel_vals.R
neg_B = neg_images_channel_vals.B


# PLOTTING THE IMAGE DATA
plt.style.use('fivethirtyeight')
light_blue="#ADD8E6"
light_red='#FFCCCB'

fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(12,10), sharey=True)
plt.subplots_adjust(hspace=0.4)

axes[0,0].set_title("Positive Images - Red")
axes[0,0].set_xlabel("Avg Pixel Value (0 - 255)",fontsize="small")
axes[0,0].set_ylabel("Count", fontsize="small")
axes[0,0].set_xlim(left=0,right=255)
axes[0,0].hist(pos_R, 20, range=(0,255), color=light_red)

axes[0,1].set_title("Positive Images - Blue")
axes[0,1].set_xlabel("Avg Pixel Value (0 - 255)",fontsize="small")
axes[0,1].set_ylabel("Count", fontsize="small")
axes[0,1].set_xlim(left=0,right=255)
axes[0,1].hist(pos_B, 20, range=(0,255), color=light_blue)

axes[1,0].set_title("Negative Images - Red")
axes[1,0].set_xlabel("Avg Pixel Value (0 - 255)",fontsize="small")
axes[1,0].set_ylabel("Count", fontsize="small")
axes[1,0].set_xlim(left=0,right=255)
axes[1,0].hist(neg_R, 20, range=(0,255), color=light_red)

axes[1,1].set_title("Negative Images - Blue")
axes[1,1].set_xlabel("Avg Pixel Value (0 - 255)",fontsize="small")
axes[1,1].set_ylabel("Count", fontsize="small")
axes[1,1].set_xlim(left=0,right=255)
axes[1,1].hist(neg_B, 20, range=(0,255), color=light_blue)

The cell-like shapes in the average negative image above show up here. The distribution of purple in the negative images appears to be bimodal, but the positive images resemble a normal distribution. When comparing to the average images above, it is important to note that these histograms are averaged across all the pixels of an image. The average images, however, are averaged across all images and show average data for each pixel.

The histograms suggest that there are two different "average" hues of purple among the negative images, while the positive images tend to average out to similar hues of purple. Since the averages of both channels in the negative and positive images appear to be in similar locations, the color distribution does not really provide a way to distiguish between the two classes consistently. The distributions may suggest something about the colors of the features/objects in the images though, which hopefully the CNN model can identify.


#### One thing that I did not do...

The histograms above are really an amalgamation of lots of pixel information. When generating the data, the R and B channels of each image were averaged across an entire image then saved. It would be interesting to investigate/generate histograms of the R and B channels for each pixel across all images. Or, even better, useful information might be available by looking at the avg. value of the R & B channels across sections of the images. These histograms would reveal more detailed information about distribution of objects in the images, which is essentially removed in the plots above because of averaging across an entire image.

By implementing a convolutional neural network, hopefully, the object level information can be extracted better than the plots created above.


# Step 3 - Model Architecture

In this section, I try a variety of model architectures to find a strong model for classifying the metastatic cancer images.

## First, some code to prepare the data for Steps 3 & 4

In [13]:
# RE-ARRANGING THE IMAGE FILES TO BE COMPATIBLE WITH tf.keras.utils.images_from_directory...

!mkdir /kaggle/working/train
!mkdir /kaggle/working/train/pos
!mkdir /kaggle/working/train/neg

# Other folders that need to be made
!mkdir /kaggle/working/design_testing
!mkdir /kaggle/working/model_tuning
!mkdir /kaggle/working/trained_model


y_info = pd.read_csv(f"{INPUT}/train_labels.csv", header=0)

for i in range(y_info.shape[0]):
    f = y_info.loc[i,"id"]
    img = io.imread(INPUT+"/train/"+f+".tif")
    if y_info.loc[i,"label"]:   # "== 1"
        io.imsave(f"{TRAIN}/pos/"+f+".jpg", img, check_contrast=False, quality=100)
        #os.remove(INPUT+"/train/"+f+".tif")
    else:
        #os.remove(INPUT+"/train/"+f+".tif")
        io.imsave(f"{TRAIN}/neg/"+f+".jpg", img, check_contrast=False, quality=100)

## Design testing plan

I'll stick to a simple model. I plan to use the "building block-style" Covolution-Convolution-Pooling pattern. Since we previously experimented with the development of neural network architecture, I am hoping to replicate a reliable NN structure from one of the example image classification models from the videos.

Laid out below are my achitecture testing plans as well as some of the thoughts I have regarding the training of my CNN.

__Design parameters and Hyperparameters__
Decisions
* I will use ReLU (hidden layers) and sigmoid (output layer) as activation functions. This is not a design parameter that I plan to vary this time.
* I will primarily use 3x3 convolutional filters
* As an optimization method, I will stick to SGD with momentum because I am most familiar with SGD

--- __*Update*__ ---
* Decided to change optimizer: SGD -> Adam
  - I was looking for ways to improve my results and Adam was suggested
  - I'm going to use the default values for the parameters that I am not familiar with: "beta_1", "beta_2", "epsilon"

Hyperparameters
* Learning rate
    * Test: 0.01 | 0.001 | 0.0001 (3 values)
* Number of epochs (i.e., how much training)

Design
* Number of [Conv-Conv-Pool] layers
    - Test: 2, 3, 4
* Number of filters to use

Potential ways to improve a struggling model
* L2 regularization
* Batch normalization

I plan to use moderate training parameters at first (e.g. learning rate -> 0.001) to experiment and narrow down some viable convolution designs.

## Step 3 - Part 1: Loading the data

In [21]:
# Loading the image data
train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN,
    validation_split=0.33,
    subset="training",
    seed=123,
    image_size=(img_dim, img_dim),
    batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN,
    validation_split=0.33,
    subset="validation",
    seed=123,
    image_size=(img_dim, img_dim),
    batch_size=batch_size)

for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

## Step 3 - Part 2: Creating functions to build model and prepare for training

I decided to create functions for the task of creating the model object each for each new design. For me, it created a cleaner setup for the process of modifying the model design and running the training procedure over and over.

Additionally, here are some decisions that I made regarding the images that are part of all model iterations:
* I normalized the images to a [0,1] scale
* I cropped the images since only the middle 32x32 pixel region of the images is used for classification

In [22]:
## Functions that take model designs and parameters and turn them into a model object

def model_creator(layers_lst, layer_design):
    model = tf.keras.Sequential()
    model.add(layers.Rescaling(1./255))
    model.add(layers.Cropping2D(cropping=((12,12),(12,12)), data_format='channels_last'))

    for (l, d) in zip(layers_lst, layer_design):
        if l == "input":
            model.add(layers.Conv2D(d["filters"], d["kernel_size"], activation='relu', padding=d["padding"], input_shape=d["input_shape"]))
        elif l == "conv":
            model.add(layers.Conv2D(d["filters"], d["kernel_size"], activation='relu', padding=d["padding"]))
        elif l == "maxpool":
            model.add(layers.MaxPool2D(d["pool_size"], strides=d["strides"]))
        elif l == "flatten":
            model.add(layers.Flatten())
        elif l == "dense":
            model.add(layers.Dense(d["size"], activation=d["activation"]))
        elif l == "output":
            model.add(layers.Dense(d["size"]))
        elif l == "dropout":
            model.add(layers.Dropout(d["rate"]))
        elif l == "norm":
            model.add(layers.BatchNormalization(axis=3))  # not 100 percent sure 3 is correct
        elif l == "conv_pool":
            model.add(layers.Conv2D(d["filters"], d["kernel_size"], strides=d["strides"], padding=d['padding']))
        else:
            raise Exception("Invalid layer provided for the model")

    return model


def model_compiler(model, opt_param, metrics):
    opt = optimizers.Adam(learning_rate=opt_param)
    model.compile(optimizer=opt,
                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=metrics)
    return model

## Step 3 - Part 3: Testing model parameters

The first code cell includes two lists that I modified to design the model each time.

The second cell allowed me to train each new design, then visualize and save information about the model.

In [23]:
layers_lst = ["input",
              "conv_pool",
              "norm",
              "conv",
              "conv",
              "maxpool",
              "norm",
              "conv",
              "conv",
              "maxpool",
              "norm",
              "flatten",
              "dense",
              "dropout",
              "dense",
              "dropout",
              "dense"
              ]
layer_design = [
    {"filters":16, "kernel_size":(7,7), "padding":"valid", "input_shape":(72,72,3)},
    {"filters":16, "kernel_size":(5,5), "strides":(2,2), "padding":"same"},
    None,
    {"filters":32, "kernel_size":(3,3), "padding":"same"},
    {"filters":32, "kernel_size":(3,3), "padding":"same"},
    {"pool_size":(2,2), "strides":(2,2)},
    None,
    {"filters":64, "kernel_size":(3,3), "padding":"same"},
    {"filters":64, "kernel_size":(3,3), "padding":"same"},
    {"pool_size":(2,2), "strides":(2,2)},
    None,
    None,
    {"size":128, "activation":'relu'},
    {"rate":0.25},
    {"size":64, "activation":'relu'},
    {"rate":0.25},
    {"size":1, "activation":'sigmoid'}]

In [24]:
# ***********  STEPS TO CREATE THE MODEL  *****************************
model = model_creator(layers_lst, layer_design)
model = model_compiler(model, 0.001, ['accuracy'])


history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=4
                    )

model.summary()

# ************** VIEWING THE OUTPUTS ***********************
print("--- Ending Accuracy ---","\n",
      f"Validation: {history.history['val_accuracy'][-1]:.2%}","\n",
      f"Training: {history.history['accuracy'][-1]:.2%}","\n")

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

# *********** SAVING THE MODEL AND RESULTS *****************
now_obj = datetime.now()
now_date = now_obj.strftime("%Y_%m_%d")
now_time = (now_obj.hour * 3600) + (now_obj.minute * 60) + now_obj.second
now = f"{now_date}_{now_time}"

filename1=f"{now}_{int(history.history['val_accuracy'][-1] * 100)}.json"
filename2=f"{now}_layers.json"
filename3=f"{now}_design.json"


with open(f"{PATH}/design_testing/{filename1}", "w") as write_file:
    model_json = model.to_json()
    json.dump(model_json, write_file)
    
with open(f"{PATH}/design_testing/{filename2}", "w") as write_file:
    json.dump(layers_lst, write_file)

with open(f"{PATH}/design_testing/{filename3}", "w") as write_file:
    json.dump(layer_design, write_file)

## Choices for my model design

After testing many designs, I have decided to use the design below. You can find information about the other designs I have tested in the "design_testing" folder. I saved JSON files with information about each design tested.

Image Modification Layers:
* Adjust pixel intensities from [0,255] to [0,1]
* Crop input images to remove 24 px from height and width.

Convolution Layers:
* Conv2D -> 7x7 kernel & 16 filters
* Conv2D -> 5x5 kernel & 16 filters
* BatchNormalization
* Conv2D -> 3x3 kernel & 32 filters
* Conv2D -> 3x3 kernel & 32 filters
* MaxPool -> 2x2 region
* BatchNormalization
* Conv2D -> 3x3 kernel & 64 filters
* Conv2D -> 3x3 kernel & 64 filters
* MaxPool -> 2x2 region
* Batch Normalization

Arificial Neural Network:
* Dense -> 128
* Dropout -> 0.2
* Dense -> 64
* Dropout -> 0.2
* Dense -> 1 (with sigmoid acitvation function)

I noticed a couple trends that led me to choose this design. Using the chosen style for convolution layers - i.e., Conv-Conv-Pool, designs with many layers were inconsistent. In early layers, large filters and strides greater than one had better results with regard to training and validation accuracy.

It seems likely, with the right additions, that deeper networks would do a better job on this task. The deep ImageNet models like LeNet and ResNet have a lot of success, but I was not able to use the `Sequential` model and include the connections that make these models successful. For the deeper designs tested, I concluded that high variance training and validation accuracy (see data at the beginning of step 4) were the result of the model complexity and ended up with a model with relatively few parameters.

Images provided are much larger than the regions used to classify them. My initial approach was to crop most of the unneeded region. Later in the testing process, the model achieved better success when less of the image was cropped out, which was paired with 5x5 and 7x7 filters and larger strides. The larger filters and strides were used in the first Conv-Conv-Pool block of my final design as a result.

#### Next up in section 4

* I will solidify all parameters of the model via hyperparameter tuning.
* I will train a model on all data to generate a final model.

# Step 4 - Results and Analysis

## Part 1 - Displaying designs tested and analyzing the CNN model

Before moving on to hyperparameter tuning, I have included a brief summary of the designs that I tested below. Then, I discuss some of the techniques used to improve the overall success of my model design (- I tested these in the previous step while testing other model design features).

In [25]:
cnn_designs = pd.read_csv(f"{INPUT2}/cnn_designs_tested.csv")
print(cnn_designs)

I introduced some normalization techniques toward the end of my design testing procedure.

First, I started introducing batch normalization into the convolution layers of the model. I noticed initial improvement compared to the similar designs that I had already tested and decided to include a batch noramlization layer in each Conv-Conv-Pool block. I did not extensively experiment with the effects of batch normalization though.

Second, I introduced dropout layers in the neural network. I tried dropout rates between 0.1 and 0.5. Designs with lots of parameters had better results with rates around 0.3, but slightly lower rates worked better for models of the size that I chose in the end. Thus, I use 0.2 as the dropout rate in both of the dropout layers.

## Part 2 - Preparations for hyperparameter tuning

In [15]:
LEARNING_RATES = [0.0001, 0.001, 0.01]
N_EPOCHS = 6

# FUNCTION TO CREATE MODEL AND TRAIN - allows me to easily test hyperparameters

def train_model(train_ds, val_ds, learning_rate, num_epochs):
    model = tf.keras.Sequential([
        layers.Rescaling(1./255),
        layers.Cropping2D(cropping=((12,12),(12,12)), data_format="channels_last"),
        layers.Conv2D(16, (7,7), activation='relu', padding='valid', input_shape=(72,72,3)),
        layers.Conv2D(16, (5,5), activation='relu', strides=(2,2), padding='same'),
        layers.BatchNormalization(axis=-1),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.Conv2D(32, (3,3), activation='relu', padding='same'),
        layers.MaxPool2D((2,2), strides=(2,2)),
        layers.BatchNormalization(axis=-1),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPool2D((2,2), strides=(2,2)),
        layers.BatchNormalization(axis=-1),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(1, activation='sigmoid')
    ])
    
    opt = optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt,
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=['accuracy'])

    results = model.fit(train_ds, verbose=0, validation_data=val_ds, epochs=num_epochs)

    return results, model

## Part 3 - Tuning hyperparameters

In [27]:
results = []

# Retrieving information so I can save model results
now_obj = datetime.now()
now_date = now_obj.strftime("%Y_%m_%d")
now_time = (now_obj.hour * 3600) + (now_obj.minute * 60) + now_obj.second
now = f"{now_date}_{now_time}"

counter=0
for eta in LEARNING_RATES:
    # Train a model with the selected learning rate
    run_results, model = train_model(train_ds, val_ds, eta, N_EPOCHS)
    if counter < 1:
        model.summary()
        counter += 1
    # Save results for plotting later
    results.append(run_results)
    # Saving model info
    filename = f"{PATH}/model_tuning/{now}_model_{eta}"
    model.save(filename)  # all other parameters default to the values I wanted

# Viewing the results
fig, axes = plt.subplots(figsize=(12,6), nrows=1, ncols=3, sharey=True)
plt.subplots_adjust(wspace=0.2)

for i in range(len(LEARNING_RATES)):
    axes[i].plot(results[i].history['accuracy'], label='accuracy')
    axes[i].plot(results[i].history['val_accuracy'], label='val_accuracy')
    axes[i].set_xlabel("Epoch")
    axes[i].set_ylim([0.5,1])
    axes[i].legend(loc="lower right")

    val_accuracy = results[i].history['val_accuracy'][-1]
    train_accuracy = results[i].history['accuracy'][-1]
    title = f"Learning Rate: {LEARNING_RATES[i]}\nTrain: {train_accuracy:.2%} - Val: {val_accuracy:.2%}"
    axes[i].set_title(title)

## Part 4 - Training final model

In [21]:
# DEBUGGING CELL - PLEASE IGNORE
!mkdir /kaggle/working/trained_model
model.save(filename)

In [18]:
full_train_ds = tf.keras.utils.image_dataset_from_directory(
    TRAIN,
    image_size=(img_dim, img_dim),
    batch_size=batch_size)

results, model = train_model(full_train_ds, None, learning_rate=0.0001, num_epochs=4)

# Retrieving information so I can save model results
now_obj = datetime.now()
now_date = now_obj.strftime("%Y_%m_%d")
now_time = (now_obj.hour * 3600) + (now_obj.minute * 60) + now_obj.second
now = f"{now_date}_{now_time}"

filename = f"{PATH}/trained_model/{now}_model"
model.save(filename)

# Viewing the training results
plt.plot(results.history['accuracy'], label='accuracy')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.ylim([0.5, 1])
info = results.history['accuracy'][-1]
plt.title(f"Final Accuracy: {info:0.2%}")

## Part 5 - Testing the model on the test dataset

#### First, some helper functions

In [32]:
def load_test_image(FILE_PATH):
    img = io.imread(FILE_PATH)
    return img

def load_batch(FILES_PATH, FILES):
    n = len(FILES)
    imgs = np.zeros((n,96,96,3), dtype=np.int8)
    for i in range(n):
        img = io.imread(str(FILES_PATH)+"/"+FILES[i])
        imgs[i,:,:,:] = img
    return imgs

def evaluate_test_images(model, FILES_PATH, img_files):
    image_ids = []
    raw_outputs = []
    results = []

    iter = 0
    while (len(img_files) - iter) > 128:
        batch_files = img_files[iter:(iter+64)]
        imgs = load_batch(TEST, batch_files)
        batch_img_ids = [f[:-4] for f in batch_files]

        iter += 64
        image_ids.extend(batch_img_ids)
        
        batch_result = model.predict(imgs)
        raw_outputs.extend(batch_result[:,0])
        batch_result = np.where(batch_result > 0.5, 1, 0)
        results.extend(batch_result[:,0])
    

    batch_files = img_files[iter:]
    imgs = load_batch(TEST, batch_files)
    batch_img_ids = [f[:-4] for f in batch_files]

    image_ids.extend(batch_img_ids)
    
    batch_result = model.predict(imgs)
    raw_outputs.extend(batch_result[:,0])
    batch_result = np.where(batch_result > 0.5, 1, 0)
    results.extend(batch_result[:,0])

    return pd.DataFrame({'id':image_ids, 'label':results, 'raw_output':raw_outputs})

# TESTING "load_batch" FUNCTION
sample_batch = load_batch(TEST, TEST_LOCS[0:64])
print(sample_batch.shape)

#### Now, loading the model and evaluating the test images

Location of trained model: `/kaggle/working/trained_model/...`

A function was prepared above to assist with loading and evaluating each test image.

In [25]:
# DEBUGGING CELL - PLEASE IGNORE
working_files = os.listdir(PATH)
print(len(working_files))
for i in range(6):
    print(working_files[i])
    
model_file = os.listdir("/kaggle/working/trained_model")
print(model_file)

In [33]:
model = tf.keras.models.load_model(f"{PATH}/trained_model/2022_08_08_8081_model")

model.summary()

results = evaluate_test_images(model, TEST, TEST_LOCS)

results.head(20)

# SAVING THE TESTING RESULTS AND COUNTING THE NUMBER OF 1s/0s
test_counts = results.loc[:,"label"].value_counts()
print(test_counts)

results.loc[:,["id","label"]].to_csv(f"{PATH}/submission.csv", index=False)

## Part 6 - Model optimization and discussion of testing

### Hyperparameter optimization

I used similar procedure for hyperparemeter testing that I did for testing model designs. 

I divided up the training images into training and validation sets. I trained the model (with my chosen design) using three learning rates and 5 epochs each time. I assessed the training process by plotting the training and validation accuracy across the 5 epochs.

Analayzing the training graphs, I selected a learning rate that appeared to improve training and validation accuracy consistently. I was looking for a training procedure that would yield gradual improvements toward an optimal solution rather than high accuracy results that might overfit the training and validation data.

### Analysis of model testing

I was able to train a model that achieved greater than 80% training and validation accuracy using a learning rate of 0.001 and 4 epochs. This seemed to be the best combination of parameters to use for training.

After preparing the model on all the training data, the model achieved greater than 90% accuracy during the last epoch. However, when evaluating on the test data, the overall score was very low -- approx. 0.51.

My initial idea to explain this low test performance is overfitting to the images that I have been experimenting and training with over and over. I have not worked with many datasets as large as this, so I was unsure about how much data to use at each stage of model development. Normally, it is standard to use as much data as possible. Images of cells, in my opinion, are not very complex though and tend to be repetitive, so I was consistently concerned about overfitting and model complexity.

# Step 5 - Conclusion

I prematurely ended my efforts to develop a great model. I wanted to spend a reasonable amount of time and set reasonable expectations, so I stopped tweaking my design even though I knew I could probably achieve better. For my first significant deep learning project, it seemed too drastic to try to re-invent a model like ResNet or LeNet. As a result, the testing performance of my model was not great (see image with my Kaggle results - "parkerdunn_kaggle_submission.jpg").

It was a great experience despite the poor results. This was one of the largest ML projects that I pursued so far with regard to size of the model, size of the data, and time invested into model development. It was a valuable lesson about the the model development process, and I know that I will achieve better models in the future as I leave myself more time to work on tuning and design testing. I found myself at times struggling at non-essential steps do to lack of experience and familiarity with this process. For example, I knew I would not be able to load all of the images into a table format at once (I tried anyway, but was pretty sure it wouldn't work), but it took me some time to figure out a strong way to work with the images. I feel like I have a much better sense now of how to prepare and work with data for a deep learning project.

I gained a much better appreciation for regularization techniques during this project. In previous work, I hadn't worked with a dataset and deep model that demonstrated the importance of dropout layers. In this project, it was evident during the training and valudation (while testing designs) that dropout layers did help with overfitting.

Another big lesson that I learned was to setup consistent ways to visualize model design and performance. In addition to realizing the ease and value of plotting performance, I prohibited my progress at times by not double-checking my model designs with `model.summary()`. It seemes to me that there is no reason not to use it each time a model is created for testing, and I will certainly use it in the future to avoid designing a model in an unintended manner.
